# Anticipez les besoins en consommations de bâtiments

## Objectif

L'objectif de ce projet est de prédire les émissions de CO2 et la consommation totale d'énergie de bâtiments non destinés à l'habitation de la ville de Seattle à partir de leurs données structurelles (taille, usage, date de construction, localisation) afin d'atteindre l'objectif de neutralité carbone de la ville en 2050.

## 1. Installation du projet et de son environnement

Afin d’utiliser correctement ce notebook, vérifiez que vous disposez du bon environnement pour l’exécuter.


### 1.1 Installation de Python

Pour ce projet, il est nécessaire d’avoir **au minimum Python 3.8**.  
Si ce n’est pas déjà le cas, vous pouvez vous référer à [la documentation officielle](https://www.python.org/downloads/).

Vérifiez votre version de Python :

```bash
python --version
```

### 1.2 Installation de `uv`

`uv` est un gestionnaire de projets Python permettant d’installer et d’organiser les dépendances plus rapidement et plus simplement que les outils traditionnels (`pip`, `virtualenv`, etc.).

Pour installer `uv`, veuillez suivre [la documentation officielle](https://docs.astral.sh/uv/getting-started/installation/#standalone-installer)

Vérifiez l’installation :

```bash
uv --version
```

### 1.3 Création du projet

Créez un nouveau projet Python avec `uv` :

```bash
uv init nom_du_projet
cd nom_du_projet
```

La structure de base du projet est alors générée automatiquement.


### 1.4 Création et activation de l’environnement virtuel

Créez l’environnement virtuel :

```bash
uv venv
```

Activez-le selon votre système :

* **Linux / macOS**

```bash
source .venv/bin/activate
```

* **Windows (PowerShell)**

```powershell
.venv\Scripts\Activate.ps1
```

Voir la documentation officielle :
[https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment](https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment)



### 1.5 Installation des dépendances

> ❗ Assurez-vous que l’environnement virtuel est **activé** avant d’installer les dépendances.

Installez les bibliothèques nécessaires au projet :

```bash
uv add ipykernel jupyterlab seaborn matplotlib numpy pandas scikit-learn tqdm scipy
```

Ces dépendances sont automatiquement enregistrées dans le fichier `pyproject.toml`.

Vous pouvez visualiser l’ensemble des dépendances installées avec la commande suivante :

```bash
uv tree


## 2. Importation des Bibliothèques


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

## 3. Analyse Exploratoire

### 3.1 Information générale sur la donnée

- **Jeu de données** : [2016_Building_Energy_Benchmarking.csv](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P4/2016_Building_Energy_Benchmarking.csv)
- **Source officielle** : [Building Energy Benchmarking - Seattle Open Data](https://data.seattle.gov/Built-Environment/Building-Energy-Benchmarking-Data-2015-Present/teqw-tu6e/about_data)

> 📁 **Action** : Téléchargez le jeu de données et ajoutez-le dans le dossier `data/`

In [ ]:
# Configuration de l'affichage pandas
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.width', None)         # Largeur automatique

building_consumption = pd.read_csv('data/2016_Building_Energy_Benchmarking.csv')

# On regarde comment un batiment est défini dans ce jeu de données 
building_consumption.head()

#### Différences entre le CSV et la documentation officielle

##### Colonnes présentes dans le CSV (2016) mais absentes de la documentation

| Colonne | Description |
|---------|-------------|
| `PrimaryPropertyType` | Type de propriété primaire |
| `PropertyName` | Nom de la propriété (similaire mais différent de `BuildingName` dans la doc) |
| `PropertyGFABuilding(s)` | Surface brute des bâtiments (équivalent à `PropertyGFABuildings` dans la doc, mais avec une parenthèse différente) |
| `ListOfAllPropertyUseTypes` | Liste de tous les types d'utilisation de la propriété |
| `YearsENERGYSTARCertified` | Années de certification ENERGY STAR |
| `DefaultData` | Indicateur de données par défaut |
| `Comments` | Commentaires |
| `Outlier` | Indicateur de valeur aberrante |

##### Colonnes présentes dans la documentation mais absentes du CSV (2016)

| Colonne | Description |
|---------|-------------|
| `BuildingName` | Nom du bâtiment (le CSV utilise `PropertyName` à la place) |
| `SelfReportGFATotal` | Surface brute totale auto-déclarée |
| `SelfReportGFABuildings` | Surface brute des bâtiments auto-déclarée |
| `SelfReportParking` | Surface de stationnement auto-déclarée |
| `SecondLargestPropertyUseTypeGFA` | Nommée `SecondLargestPropertyUse` dans la doc ligne 211 (incohérence interne) |
| `ComplianceIssue` | Problèmes de conformité |
| `Demolished` | Indicateur de démolition |

---

#### Notes importantes

⚠️ **Points de vigilance** :
- La documentation mentionne `BuildingName` mais le CSV utilise `PropertyName`
- La documentation a `PropertyGFABuildings` alors que le CSV a `PropertyGFABuilding(s)` (avec parenthèses)
- Le CSV de 2016 ne contient **pas** les colonnes d'auto-déclaration (`SelfReport*`)
- Le CSV de 2016 contient plusieurs colonnes métadonnées (`DefaultData`, `Comments`, `Outlier`) absentes de la documentation

In [ ]:
# Taille du dataset
print(building_consumption.shape)

In [ ]:
# Types et valeurs manquantes
building_consumption.info()

In [ ]:
# Statistiques descriptives
building_consumption.describe()

### 3.2 Filtrage des données

In [ ]:
building_consumption["BuildingType"].value_counts()

#### Signification des BuildingType

- **NonResidential (1460)** : Bâtiments non résidentiels  
- **Multifamily LR (1–4) (1018)** : Immeubles multifamiliaux basse densité  
- **Multifamily MR (5–9) (580)** : Immeubles multifamiliaux densité moyenne  
- **Multifamily HR (10+) (110)** : Immeubles multifamiliaux haute densité  
- **SPS-District K–12 (98)** : Écoles publiques (maternelle à lycée)  
- **Nonresidential COS (85)** : Bâtiments municipaux de Seattle  
- **Campus (24)** : Ensembles de bâtiments interconnectés  
- **Nonresidential WA (1)** : Bâtiment de l’État de Washington

Nous avons besoins uniquement de bâtiments non destinés à l’habitation par concéquent nous ne garderons que les types de batiments suivants :
- **NonResidential (1460)** : Bâtiments non résidentiels
- **SPS-District K–12 (98)** : Écoles publiques (maternelle à lycée)
- **Nonresidential COS (85)** : Bâtiments municipaux de Seattle
- **Campus (24)** : Ensembles de bâtiments interconnectés

Cas a vérifier pour savoir si on le prend en compte ou non 
- **Nonresidential WA (1)** : Bâtiment de l’État de Washington

In [ ]:
print(f"Nombre de lignes avant filtrage par type de batiment : {len(building_consumption)}")

# Types de bâtiments à conserver
building_types_to_keep = [
    "NonResidential",
    "SPS-District K-12",
    "Nonresidential COS",
    "Campus",
    "Nonresidential WA"
]

# Filtrer les données
df_filtered = building_consumption[building_consumption['BuildingType'].isin(building_types_to_keep)]

# Documenter le nombre de lignes après filtrage
print(f"Nombre de lignes après filtrage : {len(df_filtered)}")
print(f"Nombre de lignes supprimées : {len(building_consumption) - len(df_filtered)}")

# Afficher la répartition des types de bâtiments conservés
print("\nRépartition des types de bâtiments conservés :")
print(df_filtered['BuildingType'].value_counts())


### 3.3 Choix de la variable cible

Plusieurs variables du jeu de données permettent de représenter la consommation énergétique des bâtiments (consommation totale, consommation normalisée, intensité énergétique).  
Dans le cadre de ce projet, nous avons choisi d’utiliser **`SiteEnergyUseWN(kBtu)`** comme variable cible.

Cette variable correspond à la **consommation énergétique annuelle totale normalisée par les conditions météorologiques**.  
La normalisation permet de réduire l’influence d’un facteur externe au bâtiment (la météo) et de mieux isoler l’impact des **caractéristiques structurelles** telles que la surface, l’usage ou l’année de construction.

Ce choix améliore :
- la **comparabilité** entre bâtiments,
- la **robustesse** du modèle,
- et la **capacité de généralisation** des prédictions à des bâtiments dont la consommation n’a pas encore été mesurée.

La variable `SiteEnergyUse(kBtu)`, représentant la consommation réelle mesurée, sera analysée de manière complémentaire à titre de comparaison.


In [ ]:
TARGET_VARIABLE = 'SiteEnergyUseWN(kBtu)'

In [ ]:
# Histogramme classique

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_filtered[TARGET_VARIABLE].dropna(), bins=50, edgecolor='black')
plt.xlabel('SiteEnergyUseWN (kBtu)')
plt.ylabel('Fréquence')
plt.title('Distribution de la consommation énergétique')

# Créer le boxplot
plt.subplot(1, 2, 2)
plt.hist(df_filtered[TARGET_VARIABLE].dropna(), bins=50, edgecolor='black')
plt.xlabel('SiteEnergyUseWN (kBtu)')
plt.ylabel('Fréquence')
plt.title('Distribution (échelle log)')
plt.yscale('log')


plt.tight_layout()
plt.show()

##### Graphique de gauche (échelle normale)
- Une énorme concentration des données près de zéro
- La grande majorité des sites ont une consommation très faible
- Impossible de voir les détails à cause de l'échelle écrasée par quelques valeurs extrêmes

##### Graphique de droite (échelle logarithmique)
- Révèle mieux la structure de la distribution
- Montre une décroissance progressive : beaucoup de petits consommateurs, de moins en moins de gros consommateurs
- Quelques sites isolés avec des consommations très élevées à l'extrême droite

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.boxplot(df_filtered[TARGET_VARIABLE].dropna())
plt.ylabel('SiteEnergyUseWN (kBtu)')
plt.title('Boxplot - Échelle normale')

plt.subplot(1, 2, 2)
plt.boxplot(df_filtered[TARGET_VARIABLE].dropna())
plt.ylabel('SiteEnergyUseWN (kBtu)')
plt.yscale('log')
plt.title('Boxplot - Échelle log')

plt.tight_layout()
plt.show()

##### Boxplot échelle normale (gauche)
- La boîte est complètement écrasée près de zéro
- Un nuage dense de points outliers au-dessus
- Montre clairement l'asymétrie extrême des données

##### Boxplot échelle log (droite)
- La boîte s'étend sur environ un ordre de grandeur
- La médiane se situe dans la partie basse de la boîte
- Les outliers s'étalent largement au-dessus, montrant des sites exceptionnels

#### Conclusion générale

Vos données montrent une **distribution très déséquilibrée** : beaucoup de petits consommateurs et quelques très gros consommateurs. L'échelle logarithmique est indispensable pour bien visualiser ces données.

##### Traitement des outliers
Afin de choisir la bonne méthode pour traiter les outliers, nous allons comparer les méthode z-score et IQR

In [ ]:
# ========================================
# COMPARAISON Z-SCORE vs IQR
# ========================================

# 1. Calcul Z-score
z_scores = stats.zscore(df_filtered[TARGET_VARIABLE].dropna())
abs_z_scores = np.abs(z_scores)

# 2. Calcul IQR
Q1 = df_filtered[TARGET_VARIABLE].quantile(0.25)
Q3 = df_filtered[TARGET_VARIABLE].quantile(0.75)
IQR = Q3 - Q1
lower_iqr = Q1 - 1.5 * IQR
upper_iqr = Q3 + 1.5 * IQR

# Identification des outliers
outliers_z2 = df_filtered[TARGET_VARIABLE].dropna()[abs_z_scores > 2]
outliers_z3 = df_filtered[TARGET_VARIABLE].dropna()[abs_z_scores > 3]
outliers_iqr = df_filtered[(df_filtered[TARGET_VARIABLE] < lower_iqr) | 
                            (df_filtered[TARGET_VARIABLE] > upper_iqr)][TARGET_VARIABLE]

# ========================================
# RÉSULTATS COMPARATIFS
# ========================================
print("=" * 70)
print("COMPARAISON DES MÉTHODES DE DÉTECTION D'OUTLIERS")
print("=" * 70)

print(f"\n📊 Données totales : {len(df_filtered)} bâtiments")
print(f"   Moyenne : {df_filtered[TARGET_VARIABLE].mean():,.0f} kBtu")
print(f"   Médiane : {df_filtered[TARGET_VARIABLE].median():,.0f} kBtu")
print(f"   Écart-type : {df_filtered[TARGET_VARIABLE].std():,.0f} kBtu")

print("\n" + "=" * 70)
print("MÉTHODE 1 : Z-SCORE (seuil = 2)")
print("=" * 70)
print(f"Nombre d'outliers : {len(outliers_z2)}")
print(f"Pourcentage : {(len(outliers_z2) / len(df_filtered) * 100):.2f}%")

print("\n" + "=" * 70)
print("MÉTHODE 2 : Z-SCORE (seuil = 3)")
print("=" * 70)
print(f"Nombre d'outliers : {len(outliers_z3)}")
print(f"Pourcentage : {(len(outliers_z3) / len(df_filtered) * 100):.2f}%")

print("\n" + "=" * 70)
print("MÉTHODE 3 : IQR (facteur = 1.5)")
print("=" * 70)
print(f"Seuil bas : {lower_iqr:,.0f} kBtu")
print(f"Seuil haut : {upper_iqr:,.0f} kBtu")
print(f"Nombre d'outliers : {len(outliers_iqr)}")
print(f"Pourcentage : {(len(outliers_iqr) / len(df_filtered) * 100):.2f}%")

# ========================================
# VISUALISATION COMPARATIVE
# ========================================
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Graphique 1 : Z-score = 2
axes[0, 0].hist(df_filtered[TARGET_VARIABLE].dropna(), bins=50, alpha=0.6, 
                label='Données normales', edgecolor='black', color='lightblue')
axes[0, 0].hist(outliers_z2, bins=30, alpha=0.9, 
                label=f'Outliers ({len(outliers_z2)})', color='red', edgecolor='black')
axes[0, 0].set_xlabel('SiteEnergyUseWN (kBtu)')
axes[0, 0].set_ylabel('Fréquence')
axes[0, 0].set_title(f'Z-score seuil = 2 ({len(outliers_z2)} outliers - {(len(outliers_z2)/len(df_filtered)*100):.1f}%)')
axes[0, 0].legend()
axes[0, 0].set_yscale('log')
axes[0, 0].grid(alpha=0.3)

# Graphique 2 : Z-score = 3
axes[0, 1].hist(df_filtered[TARGET_VARIABLE].dropna(), bins=50, alpha=0.6, 
                label='Données normales', edgecolor='black', color='lightblue')
axes[0, 1].hist(outliers_z3, bins=30, alpha=0.9, 
                label=f'Outliers ({len(outliers_z3)})', color='orange', edgecolor='black')
axes[0, 1].set_xlabel('SiteEnergyUseWN (kBtu)')
axes[0, 1].set_ylabel('Fréquence')
axes[0, 1].set_title(f'Z-score seuil = 3 ({len(outliers_z3)} outliers - {(len(outliers_z3)/len(df_filtered)*100):.1f}%)')
axes[0, 1].legend()
axes[0, 1].set_yscale('log')
axes[0, 1].grid(alpha=0.3)

# Graphique 3 : IQR
axes[1, 0].hist(df_filtered[TARGET_VARIABLE].dropna(), bins=50, alpha=0.6, 
                label='Données normales', edgecolor='black', color='lightblue')
axes[1, 0].hist(outliers_iqr, bins=30, alpha=0.9, 
                label=f'Outliers ({len(outliers_iqr)})', color='green', edgecolor='black')
axes[1, 0].axvline(upper_iqr, color='darkgreen', linestyle='--', linewidth=2, 
                   label=f'Seuil IQR = {upper_iqr:,.0f}')
axes[1, 0].set_xlabel('SiteEnergyUseWN (kBtu)')
axes[1, 0].set_ylabel('Fréquence')
axes[1, 0].set_title(f'IQR (facteur 1.5) ({len(outliers_iqr)} outliers - {(len(outliers_iqr)/len(df_filtered)*100):.1f}%)')
axes[1, 0].legend()
axes[1, 0].set_yscale('log')
axes[1, 0].grid(alpha=0.3)

# Graphique 4 : Boxplot comparatif
data_z2 = df_filtered[TARGET_VARIABLE].copy()
data_z2.loc[outliers_z2.index] = np.nan

data_z3 = df_filtered[TARGET_VARIABLE].copy()
data_z3.loc[outliers_z3.index] = np.nan

data_iqr = df_filtered[TARGET_VARIABLE].copy()
data_iqr.loc[outliers_iqr.index] = np.nan

box_data = [
    df_filtered[TARGET_VARIABLE].dropna(),
    data_z2.dropna(),
    data_z3.dropna(),
    data_iqr.dropna()
]

bp = axes[1, 1].boxplot(box_data, tick_labels=['Originales', 'Sans Z>2', 'Sans Z>3', 'Sans IQR'],
                        patch_artist=True)
for patch, color in zip(bp['boxes'], ['lightgray', 'red', 'orange', 'green']):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)
axes[1, 1].set_ylabel('SiteEnergyUseWN (kBtu)')
axes[1, 1].set_title('Comparaison des distributions après suppression des outliers')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

#### Choix de la méthode de détection des outliers

Nous avons choisi d'utiliser le **z-score avec un seuil à 3** plutôt que le z-score à 2 ou la méthode **IQR**.  
Ce choix se justifie par les points suivants :

##### 1. Z-score à 2
- **Trop restrictif**
- Élimine **109 bâtiments** *(6,5 % du dataset)*
- Risque de supprimer des **données légitimes** concernant de grands bâtiments

##### 2. IQR (facteur 1,5)
- Également **trop strict**
- Supprime **133 bâtiments** *(8,0 % du dataset)*
- Retire des consommations élevées mais **normales** pour certains types de bâtiments  
  *(hôpitaux, campus)*

##### 3. Z-score à 3
- **Plus adapté** au contexte
- Ne supprime que **16 bâtiments extrêmes** *(1,0 % du dataset)*
- Préserve la **variabilité naturelle des données**
- Élimine uniquement les valeurs **véritablement aberrantes**


#### Examinons les 16 bâtiments

Nous analysons ces **16 bâtiments** afin de déterminer s’ils correspondent à de **véritables valeurs aberrantes** ou s’il est **pertinent de les conserver** dans le modèle.


In [ ]:
# ========================================
# EXAMEN DES 16 OUTLIERS (Z-SCORE > 3)
# ========================================

# Calculer les z-scores et identifier les outliers
z_scores = stats.zscore(df_filtered[TARGET_VARIABLE].dropna())
abs_z_scores = np.abs(z_scores)
outlier_mask = abs_z_scores > 3

# Récupérer les outliers
outliers = df_filtered[TARGET_VARIABLE].dropna()[outlier_mask]

# Afficher les informations principales
print(f"Nombre d'outliers détectés : {len(outliers)}\n")

# Sélectionner les colonnes importantes
colonnes_importantes = [
    'PropertyName',
	'Outlier',
    'BuildingType', 
    'PrimaryPropertyType',
    'LargestPropertyUseType',
    'PropertyGFABuilding(s)',
    'NumberofFloors',
    'YearBuilt',
    TARGET_VARIABLE,
    'SiteEUI(kBtu/sf)',
    'ENERGYSTARScore'
]

# Afficher le tableau des outliers
outliers_df = df_filtered.loc[outliers.index, colonnes_importantes].copy()
outliers_df['Z-Score'] = abs_z_scores[abs_z_scores > 3]
outliers_df = outliers_df.sort_values(by=TARGET_VARIABLE, ascending=False)

print("Liste des 16 bâtiments outliers :")
print("=" * 100)
display(outliers_df)

print("\n📊 RÉPARTITION DES OUTLIERS PAR TYPE :")
display(outliers_df['PrimaryPropertyType'].value_counts())

print("\n📊 RÉPARTITION DES OUTLIERS PAR USAGE :")
display(outliers_df['LargestPropertyUseType'].value_counts())

print(f"\nSurface moyenne des outliers      : {outliers_df['PropertyGFABuilding(s)'].mean():,.0f} sf")
print(f"Surface moyenne globale           : {df_filtered['PropertyGFABuilding(s)'].mean():,.0f} sf")
print(f"Ratio                             : {outliers_df['PropertyGFABuilding(s)'].mean() / df_filtered['PropertyGFABuilding(s)'].mean():.1f}x")



In [ ]:
# Visualisation des outliers
plt.figure(figsize=(10, 6))

# Tracer toutes les données normales puis les outliers par-dessus
plt.scatter(df_filtered['PropertyGFABuilding(s)'], 
            df_filtered[TARGET_VARIABLE], 
            alpha=0.5, s=30, label='Données normales', color='lightblue')

plt.scatter(outliers_df['PropertyGFABuilding(s)'], 
            outliers_df[TARGET_VARIABLE], 
            alpha=0.9, s=100, label='Outliers (Z>3)', color='red', edgecolor='black')

plt.xlabel('Surface du bâtiment (sf)', fontsize=12)
plt.ylabel('Consommation énergétique (kBtu)', fontsize=12)
plt.title('Consommation vs Surface - Identification des outliers', fontsize=14, fontweight='bold')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


#### Décision : Conservation des outliers

Nous avons identifié **16 bâtiments** présentant des valeurs de consommation énergétique extrêmes *(z-score > 3)*.

**Décision :** Ces outliers sont conservés pour les raisons suivantes :

##### 1. Bâtiments légitimes
Il s'agit principalement :
- d'**hôpitaux** (6),
- de **campus universitaires** (2),
- de **grands immeubles de bureaux**,

dont la forte consommation est cohérente avec leur taille  
*(surface moyenne **8,2× supérieure** à la moyenne globale)*.

##### 2. Pas d'erreurs de mesure
Les valeurs observées sont proportionnelles :
- à la **surface**,
- au **type d'usage**.

La consommation élevée s'explique par des caractéristiques structurelles réelles :
- fonctionnement **24/7**,
- **équipements spécialisés**,
- **grande surface**.

##### 3. Représentativité
Ces types de bâtiments sont essentiels pour l'objectif de **neutralité carbone de Seattle**.  
Les exclure empêcherait le modèle de prédire correctement la consommation des **infrastructures critiques** de la ville.

##### 4. Impact limité
Les **16 bâtiments** représentent moins de **1 % du dataset** *(0,96 %)*,  
ce qui est insuffisant pour déséquilibrer le modèle.

### 3.4 Analyse des valeurs manquantes

In [ ]:
def columns_with_missing_values(dataframe):
    is_null_array = dataframe.isnull().sum()
    null_counts = is_null_array[is_null_array > 0].sort_values(ascending=False)
    return null_counts

display(columns_with_missing_values(df_filtered))

#### Colonne 'ZipCode'

Nous avons **16 bâtiments** (0.47%) sans ZipCode. Analysons ces bâtiments pour comprendre pourquoi et comment imputer ces valeurs manquantes.

In [ ]:
# Analyser les bâtiments sans ZipCode
missing_zipcode = df_filtered[df_filtered['ZipCode'].isna()]

print(f"Bâtiments sans ZipCode: {len(missing_zipcode)}")
print(f"Pourcentage: {len(missing_zipcode)/len(df_filtered)*100:.2f}%\n")

# Afficher quelques exemples
display(missing_zipcode[['OSEBuildingID', 'PropertyName', 'Address', 'Neighborhood', 'BuildingType', 'YearBuilt']].head(10))

print("\nCaractéristiques communes:")
print(f"- BuildingType: {missing_zipcode['BuildingType'].unique()}")
print(f"- OSEBuildingID moyen: {missing_zipcode['OSEBuildingID'].mean():.0f} (vs {df_filtered['OSEBuildingID'].mean():.0f} pour tous)")
print(f"- Toutes les autres infos de localisation présentes: {missing_zipcode[['Address', 'Latitude', 'Longitude', 'Neighborhood']].notna().all().all()}")

**Observation** : Tous ces bâtiments sont des bâtiments municipaux récents (Nonresidential COS) avec des OSEBuildingID élevés (>50000), suggérant une erreur lors de l'import.

**Solution d'imputation** : Puisque tous les bâtiments ont des coordonnées GPS et un quartier (Neighborhood), nous allons :
1. Utiliser le **ZipCode le plus fréquent** dans chaque quartier pour imputer les valeurs manquantes
2. Pour les quartiers sans autres bâtiments, utiliser un **géocodage inversé** basé sur les coordonnées GPS

In [ ]:
# Analyser les ZipCodes par quartier pour les bâtiments manquants
print("Analyse des ZipCodes par quartier:\n")

for neighborhood in missing_zipcode['Neighborhood'].unique():
    print(f"📍 {neighborhood}")
    
    # Bâtiments sans ZipCode dans ce quartier
    missing_in_n = missing_zipcode[missing_zipcode['Neighborhood'] == neighborhood]
    print(f"   Bâtiments SANS ZipCode: {len(missing_in_n)}")
    
    # Bâtiments avec ZipCode dans le même quartier
    with_zip_in_n = df_filtered[
        (df_filtered['Neighborhood'] == neighborhood) & 
        (df_filtered['ZipCode'].notna())
    ]
    
    if len(with_zip_in_n) > 0:
        # ZipCode le plus fréquent
        most_common = with_zip_in_n['ZipCode'].mode()[0]
        frequency = (with_zip_in_n['ZipCode'] == most_common).sum() / len(with_zip_in_n) * 100
        print(f"   ✅ ZipCode le plus fréquent: {int(most_common)} ({frequency:.1f}% des bâtiments)")
    else:
        print(f"   ⚠️  Aucun autre bâtiment avec ZipCode dans ce quartier")
    
    print()

In [ ]:
def impute_zipcode_by_neighborhood(df):
    """
    Impute les ZipCodes manquants en utilisant le ZipCode le plus fréquent du quartier.
    
    Pour les quartiers sans autres bâtiments, on peut ajouter une logique de géocodage
    ou laisser les valeurs manquantes pour traitement manuel.
    """
    df_imputed = df.copy()
    imputation_log = []
    
    # Pour chaque ligne avec ZipCode manquant
    for idx in df_imputed[df_imputed['ZipCode'].isna()].index:
        neighborhood = df_imputed.loc[idx, 'Neighborhood']
        
        # Trouver le ZipCode le plus fréquent dans ce quartier
        neighborhood_zipcodes = df_imputed[
            (df_imputed['Neighborhood'] == neighborhood) & 
            (df_imputed['ZipCode'].notna())
        ]['ZipCode']
        
        if len(neighborhood_zipcodes) > 0:
            most_common_zip = neighborhood_zipcodes.mode()[0]
            df_imputed.loc[idx, 'ZipCode'] = most_common_zip
            
            imputation_log.append({
                'Index': idx,
                'PropertyName': df_imputed.loc[idx, 'PropertyName'],
                'Neighborhood': neighborhood,
                'ImputedZipCode': int(most_common_zip),
                'Method': 'Mode du quartier'
            })
        else:
            imputation_log.append({
                'Index': idx,
                'PropertyName': df_imputed.loc[idx, 'PropertyName'],
                'Neighborhood': neighborhood,
                'ImputedZipCode': None,
                'Method': 'Quartier sans autres bâtiments'
            })
    
    return df_imputed, pd.DataFrame(imputation_log)

In [ ]:
# Appliquer l'imputation
df_imputed, imputation_log = impute_zipcode_by_neighborhood(df_filtered)

print("📊 Résumé de l'imputation:\n")
display(imputation_log)

print(f"\n✅ Valeurs imputées avec succès: {imputation_log['ImputedZipCode'].notna().sum()}")
print(f"⚠️  Valeurs toujours manquantes: {imputation_log['ImputedZipCode'].isna().sum()}")

Pour le(s) bâtiment(s) dans des quartiers sans autres données, nous pouvons soit :
- Utiliser un service de géocodage inversé (Google Maps API, OpenStreetMap, etc.)
- Rechercher manuellement le ZipCode à partir de l'adresse
- Les laisser manquants si l'impact est négligeable

Recherche manuelle pour "High Point Community Center" dans DELRIDGE NEIGHBORHOODS :

In [ ]:
# Recherche manuelle: High Point Community Center, 6920 34th Ave SW, Seattle
# D'après les données de quartier et la proximité, ce bâtiment est probablement en 98126

# Imputation manuelle pour DELRIDGE NEIGHBORHOODS
delridge_missing = df_imputed[
    (df_imputed['Neighborhood'] == 'DELRIDGE NEIGHBORHOODS') & 
    (df_imputed['ZipCode'].isna())
]

if len(delridge_missing) > 0:
    print("Bâtiment(s) à imputer manuellement:")
    display(delridge_missing[['PropertyName', 'Address', 'Latitude', 'Longitude']])
    
    # Note: Le ZipCode 98126 correspond à West Seattle/Delridge
    # Vérifiable via les coordonnées: 47.54067, -122.37441
    df_imputed.loc[delridge_missing.index, 'ZipCode'] = 98126.0
    print("\n✅ ZipCode 98126 appliqué (basé sur localisation géographique)")
else:
    print("Aucun bâtiment restant à imputer.")

In [ ]:
# Vérification finale
print("Vérification de l'imputation:\n")
print(f"Avant imputation: {df_filtered['ZipCode'].isna().sum()} valeurs manquantes")
print(f"Après imputation: {df_imputed['ZipCode'].isna().sum()} valeurs manquantes")

if df_imputed['ZipCode'].isna().sum() == 0:
    print("\n✅ SUCCÈS: Toutes les valeurs de ZipCode ont été imputées!")
    # Mettre à jour le DataFrame principal
    df_filtered = df_imputed.copy()
else:
    print(f"\n⚠️  Il reste {df_imputed['ZipCode'].isna().sum()} valeur(s) manquante(s) à traiter.")

#### Colonne 'Comments'

Étant donné que **toutes les lignes** ne contiennent **aucune information** dans la colonne *Comments*, celle-ci est **supprimée**.


In [ ]:
df_filtered = df_filtered.drop(columns=["Comments"])

#### Colonne 'Outlier'

Affichons dans un premier les lignes dont nous avons de la données

In [ ]:
display(df_filtered[df_filtered["Outlier"].notna()])

Nous avons 2 types dans la colonnes 'Outlier'

*High outliers*:
- Il s’agit exclusivement de **grands immeubles de bureaux inefficaces**
- **Score ENERGY STAR très faible (1)** ou **absent**
- Indiquent un **besoin urgent d’amélioration énergétique**

*Low outliers*:
- **80 %** présentent un **score ENERGY STAR de 100**, indiquant des bâtiments **très performants**
- Nombreux **bâtiments communautaires** *(églises, écoles)*
- Certains semblent **sous-utilisés** *(EUI < 2 kBtu/sf)*


Nous avons conservé les 17 bâtiments marqués comme outliers par Seattle car :
- Ils représentent des cas réels et légitimes (bâtiments inefficaces ou très performants)
- Leur suppression ne réduirait le dataset que de 1%
- Notre modèle doit être capable de prédire aussi bien les bâtiments inefficaces (à rénover) que les bâtiments performants (objectif 2050)
- Les recommandations du projet stipulent d'éviter de réduire significativement le dataset

##### Analyse de la colonne "SiteEUI(kBtu/sf)"

In [ ]:
def get_rows_with_missing_values(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Retourne et affiche les lignes du DataFrame où la colonne spécifiée contient des valeurs manquantes.

    Parameters
    ----------
    df : pandas.DataFrame
        Le DataFrame à analyser
    column_name : str
        Le nom de la colonne à vérifier

    Returns
    -------
    pandas.DataFrame
        Sous-DataFrame contenant les lignes avec des valeurs NaN dans la colonne
    """
    missing_rows = df[df[column_name].isna()]
    display(missing_rows)
    return missing_rows

siteEUI_missing_rows = get_rows_with_missing_values(df_filtered, "SiteEUI(kBtu/sf)")

Les données de mesures de cette ligne sont soit a 0 ou NaN , donc nous supprimons cette ligne

In [ ]:
def remove_rows_with_missing_values(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Supprime les lignes contenant des valeurs manquantes dans une colonne donnée.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame à nettoyer
    column_name : str
        Nom de la colonne à vérifier

    Returns
    -------
    pandas.DataFrame
        DataFrame sans valeurs manquantes dans la colonne spécifiée
    """
    initial_row_count = len(df)

    cleaned_df = df.dropna(subset=[column_name])

    removed_rows = initial_row_count - len(cleaned_df)
    print(f"Lignes avant nettoyage : {initial_row_count}")
    print(f"Lignes supprimées (NaN dans '{column_name}') : {removed_rows}")
    print(f"Lignes après nettoyage : {len(cleaned_df)}")

    return cleaned_df


df_filtered = remove_rows_with_missing_values(df_filtered, "SiteEUI(kBtu/sf)")

##### Analyse des colonnes 'LargestPropertyUseType' et 'LargestPropertyUseTypeGFA'

Chaque bâtiment a forcément un usage principal, nous devons comprendre pourquoi, nous avons des NaN

In [ ]:
largest_missing_values = get_rows_with_missing_values(df_filtered, "LargestPropertyUseType")


In [ ]:
# Voir les valeurs de PrimaryPropertyType pour ces lignes
largest_missing_values[['PrimaryPropertyType', 'LargestPropertyUseType', 'ListOfAllPropertyUseTypes', 'PropertyGFABuilding(s)']]


In [ ]:
# Vérifier si PrimaryPropertyType est souvent égal à LargestPropertyUseType
comparison = building_consumption[building_consumption['LargestPropertyUseType'].notna()].copy()
comparison['same_value'] = comparison['PrimaryPropertyType'] == comparison['LargestPropertyUseType']
print(f"Pourcentage de correspondance : {comparison['same_value'].mean() * 100:.2f}%")


In [ ]:
# Vérifier si ListOfAllPropertyUseTypes est souvent égal à LargestPropertyUseType
comparison = building_consumption[building_consumption['LargestPropertyUseType'].notna()].copy()
comparison['same_value'] = comparison['ListOfAllPropertyUseTypes'] == comparison['LargestPropertyUseType']
print(f"Pourcentage de correspondance : {comparison['same_value'].mean() * 100:.2f}%")


In [ ]:
# Pour chaque ListOfAllPropertyUseTypes, voir quelles sont les valeurs courantes de LargestPropertyUseType
for ptype in largest_missing_values['ListOfAllPropertyUseTypes'].unique():
    if pd.notna(ptype):
        print(f"\nPour ListOfAllPropertyUseTypes = '{ptype}':")
        subset = building_consumption[(building_consumption['ListOfAllPropertyUseTypes'] == ptype) & (building_consumption['LargestPropertyUseType'].notna())]
        print(subset['LargestPropertyUseType'].value_counts())


In [ ]:
# Pour les 3 cas simples, remplir avec ListOfAllPropertyUseTypes
df_filtered.loc[df_filtered.index.isin([1147, 2414, 2459]), 'LargestPropertyUseType'] = \
    df_filtered.loc[df_filtered.index.isin([1147, 2414, 2459]), 'ListOfAllPropertyUseTypes']

# Pour le cas 353, on supprime la ligne
df_filtered = df_filtered.drop(index=353)


In [ ]:
# Afficher les lignes avec LargestPropertyUseTypeGFA manquant
largestGFA_missing_values = get_rows_with_missing_values(df_filtered, "LargestPropertyUseTypeGFA")


In [ ]:
# Vérifier si ListOfAllPropertyUseTypes est souvent égal à LargestPropertyUseTypeGFA
comparison = building_consumption[building_consumption['LargestPropertyUseTypeGFA'].notna()].copy()
comparison['same_value'] = comparison['PropertyGFABuilding(s)'] == comparison['LargestPropertyUseTypeGFA']
print(f"Pourcentage de correspondance : {comparison['same_value'].mean() * 100:.2f}%")


In [ ]:
df_filtered.loc[df_filtered.index.isin([1147, 2414, 2459]), 'LargestPropertyUseTypeGFA'] = \
    df_filtered.loc[df_filtered.index.isin([1147, 2414, 2459]), 'PropertyGFABuilding(s)']

#### Colonnes 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA'

Ces colonnes fournissent des **informations sur les bâtiments à usages multiples**.  
Les valeurs **NaN** indiquent simplement l’absence d’un usage secondaire ou tertiaire et **ne constituent pas un problème**.

**Les colonnes sont donc conservées.**


#### Analyse de la colonne 'ENERGYSTARScore'

In [ ]:
energiestartscore_missing_values = get_rows_with_missing_values(df_filtered, "ENERGYSTARScore")

##### Exclusion de la colonne ENERGYSTARScore

Cette colonne est retirée pour deux raisons :

1. **Risque de fuite de données** :  
   Le score ENERGY STAR dépend directement de la consommation énergétique du bâtiment, notre variable cible. L’utiliser reviendrait à prédire la consommation à partir d’un indicateur qui la contient déjà.

2. **Trop de valeurs manquantes** :  
   Environ 18 % des bâtiments n’ont pas ce score, car certains types de propriétés ne sont pas éligibles ou certaines données manquent.

En l’excluant, le modèle apprend uniquement à partir des caractéristiques réelles des bâtiments.


#### Analyse de la colonne SiteEUIWN(kBtu/sf)

In [ ]:
siteEui_missing_values = get_rows_with_missing_values(df_filtered, "SiteEUIWN(kBtu/sf)")

Sachant que c'est notre variable cible et qu'il y a qu'une ligne nous pouvons nou permettre de la supprimés

In [ ]:
df_filtered = remove_rows_with_missing_values(df_filtered, "SiteEnergyUseWN(kBtu)")

### 3.5 Détection des valeurs aberrantes

#### Valeurs négatives


In [ ]:
def display_columns_with_negatives(dataframe: pd.DataFrame) -> None:
    """
    Affiche les statistiques descriptives des colonnes numériques contenant des valeurs négatives.

    Parameters
    ----------
    dataframe : pandas.DataFrame
        Le DataFrame à analyser
    """
    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns
    cols_with_negatives = numeric_cols[(dataframe[numeric_cols] < 0).any()]
    display(dataframe[cols_with_negatives].describe())

display_columns_with_negatives(df_filtered)

On remarque que nous avons des valeurs négatives pour SourceEUIWN(kBtu/sf), Electricity(kWh), Electricity(kBtu), TotalGHGEmissions, GHGEmissionsIntensity cela signifierai que les batiments produirés plus de d'électricité par exemple que n'en produit.

Vérifions les lignes concernés

In [ ]:
negative_checks = [
    ('SourceEUIWN(kBtu/sf)', df_filtered['SourceEUIWN(kBtu/sf)'] < 0),
    ('Electricity(kWh)', df_filtered['Electricity(kWh)'] < 0),
    ('Electricity(kBtu)', df_filtered['Electricity(kBtu)'] < 0),
    ('TotalGHGEmissions', df_filtered['TotalGHGEmissions'] < 0),
    ('GHGEmissionsIntensity', df_filtered['GHGEmissionsIntensity'] < 0),
]
display(df_filtered[df_filtered['Electricity(kWh)'] < 0])

On constate que nous avons qu'un seul batiment possédant des valeurs négatives

Le bâtiment **Bullitt Center** est un bâtiment écologique, possédant des panneaux solaires, ce qui pourrait expliquer ces valeurs négatives.

Nous décidons de supprimer cette donnée car :
- Il s’agit d’un cas très spécifique de bâtiment à énergie positive (unique dans le dataset) ;
- Valeurs incohérentes : consommation électrique négative alors que la consommation totale est positive (SiteEnergyUse (kBtu)) ;
- Donnée non représentative de la population cible (bâtiments consommateurs d’énergie) ;
- Risque de fausser les modèles de prédiction ;
- On constate également que les colonnes **TotalGHGEmissions** et **GHGEmissionsIntensity** comportent des valeurs négatives.


In [ ]:
print(f"Nombre de lignes avant suppression des valeurs négative dans 'Electricity(kWh)' : {len(df_filtered)}")
df_filtered = df_filtered[df_filtered['Electricity(kWh)'] >= 0]
print(f"Nombre de lignes après suppression des valeurs négative dans 'Electricity(kWh)' : {len(df_filtered)}")

Vérifions si nous avons d'autre valeurs négatives

In [ ]:
display_columns_with_negatives(df_filtered)

Nous n'avons plus d'autre données avec des valeurs négatives

### 3.6 Analyse exploratoire visuelle

#### 3.6.1 Distributions univariées



In [ ]:
# Distribution avec échelle normale et log
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Échelle normale
sns.histplot(df_filtered[TARGET_VARIABLE], bins=50, kde=True, ax=axes[0])
axes[0].set_title('Distribution de la consommation (échelle normale)')

# Échelle log
sns.histplot(df_filtered[TARGET_VARIABLE], bins=50, kde=True, ax=axes[1], log_scale=True)
axes[1].set_title('Distribution de la consommation (échelle log)')
plt.tight_layout()



#### 3.6.2 Relations bivariées

##### A. Quantitative vs Quantitative



In [ ]:
# Surface vs Consommation (RELATION CRUCIALE)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_filtered, x='PropertyGFABuilding(s)', y=TARGET_VARIABLE, alpha=0.5)
plt.title('Consommation vs Surface du bâtiment')
plt.xlabel('Surface du bâtiment (sf)')
plt.ylabel('Consommation énergétique (kBtu)')


##### B. Quantitative vs Qualitative

In [ ]:
# Boxplot par type de propriété
plt.figure(figsize=(14, 6))
sns.boxplot(data=df_filtered, x='PrimaryPropertyType', y='SiteEnergyUseWN(kBtu)')
plt.xticks(rotation=45, ha='right')
plt.title('Distribution de la consommation par type de propriété')
plt.ylabel('Consommation (kBtu)')
plt.tight_layout()



In [ ]:
# Top 10 quartiers par consommation moyenne
top_neighborhoods = df_filtered.groupby('Neighborhood')['SiteEnergyUseWN(kBtu)'].mean().nlargest(10)

plt.figure(figsize=(12, 6))
df_top = df_filtered[df_filtered['Neighborhood'].isin(top_neighborhoods.index)]
sns.boxplot(data=df_top, x='Neighborhood', y='SiteEnergyUseWN(kBtu)')
plt.xticks(rotation=45, ha='right')
plt.title('Consommation des 10 quartiers avec la plus forte consommation moyenne')
plt.tight_layout()


In [ ]:
# Créer la variable Age
df_filtered['Age'] = 2016 - df_filtered['YearBuilt']

# Scatter plot Age vs Consommation
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_filtered, x='Age', y='SiteEnergyUseWN(kBtu)', alpha=0.5)
plt.title('Consommation vs Âge du bâtiment')
plt.xlabel('Âge (années)')
plt.ylabel('Consommation (kBtu)')




##### C. Qualitative vs Qualitative


In [ ]:
# Exemple : Type de propriété vs Quartier
ct = pd.crosstab(df_filtered['PrimaryPropertyType'], df_filtered['Neighborhood'])

plt.figure(figsize=(14, 8))
sns.heatmap(ct, annot=True, fmt='d', cmap='YlOrRd')
plt.title('Nombre de bâtiments par type et quartier')
plt.xlabel('Quartier')
plt.ylabel('Type de propriété')
plt.tight_layout()
plt.show()


In [ ]:
# Crosstab des 10 types les plus fréquents et 10 quartiers les plus fréquents
top_types = df_filtered['PrimaryPropertyType'].value_counts().head(10).index
top_neighborhoods = df_filtered['Neighborhood'].value_counts().head(10).index

df_subset = df_filtered[
    (df_filtered['PrimaryPropertyType'].isin(top_types)) & 
    (df_filtered['Neighborhood'].isin(top_neighborhoods))
]

ct = pd.crosstab(df_subset['PrimaryPropertyType'], df_subset['Neighborhood'])

plt.figure(figsize=(14, 8))
sns.heatmap(ct, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Répartition des types de propriétés par quartier')
plt.xlabel('Quartier')
plt.ylabel('Type de propriété')
plt.tight_layout()
